In [51]:
import os
import pytz
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from django.apps import apps as django_apps
from django.db.models.signals import post_save
from django.dispatch import receiver
from flourish_caregiver.helper_classes.utils import cohort_assigned
from flourish_caregiver.helper_classes import SequentialCohortEnrollment
from flourish_caregiver.helper_classes.schedule_dict import child_schedule_dict, caregiver_schedule_dict
from flourish_child.models import Appointment as ChildAppointment
from edc_appointment.models import Appointment
from edc_appointment.constants import NEW_APPT
from edc_visit_schedule import site_visit_schedules
from edc_base.utils import get_utcnow, age, relativedelta
from copy import copy

In [69]:
def take_off_schedule(onschedule_model_obj, offschedule_dt=get_utcnow()):
    subject_identifier = onschedule_model_obj.subject_identifier
    schedule_name = onschedule_model_obj.schedule_name
    onschedule_model = onschedule_model_obj.onschedule_model

    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
        onschedule_model=onschedule_model, name=schedule_name)

    if schedule.is_onschedule(
        subject_identifier=subject_identifier, report_datetime=get_utcnow()):
        schedule.take_off_schedule(
            subject_identifier=subject_identifier,
            schedule_name=schedule_name,
            offschedule_datetime=offschedule_dt)

In [53]:
def delete_completed_appointments(appointment_model_cls, subject_identifier,
                                  schedule_name ):
    complete_appts = appointment_model_cls.objects.filter(
        Q(schedule_name__icontains='quart') | Q(schedule_name__icontains='qt'),
        subject_identifier=subject_identifier, ).exclude(
            appt_status=NEW_APPT).values_list('visit_code', flat=True).distinct()

    new_appts = appointment_model_cls.objects.filter(
        subject_identifier=subject_identifier,
        schedule_name=schedule_name,
        visit_code__in=complete_appts)
    if new_appts.exists():
        new_appts.delete()

In [54]:
def update_onschedule_model(subject_identifier, child_subject_identifier,
                            onschedule_model, schedule_name, ):

    onschedule_model_cls = django_apps.get_model(onschedule_model)
    try:
        onschedule_model_cls.objects.get(
            subject_identifier=subject_identifier,
            schedule_name=schedule_name,
            child_subject_identifier=child_subject_identifier)
    except onschedule_model_cls.DoesNotExist:
        try:
            onschedule_obj = onschedule_model_cls.objects.get(
                subject_identifier=subject_identifier,
                schedule_name=schedule_name)
        except schedule.onschedule_model_cls.DoesNotExist:
            pass
        else:
            onschedule_obj.child_subject_identifier = child_subject_identifier
            onschedule_obj.save()

In [55]:
def put_on_schedule(onschedule_model, schedule_name,
                    subject_identifier, onschedule_datetime=None, base_appt_datetime=None, ):
    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
        onschedule_model=onschedule_model,
        name=schedule_name)

    schedule.put_on_schedule(
        subject_identifier=subject_identifier,
        onschedule_datetime=onschedule_datetime or get_utcnow(),
        base_appt_datetime=base_appt_datetime,
        schedule_name=schedule_name)

In [56]:
def put_onschedule(onschedule_datetime, subject_identifier,
                   onschedule_model, schedule_name, appointment_cls):
    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
            onschedule_model=onschedule_model,
            name=schedule_name)

    if not schedule.is_onschedule(
        subject_identifier=subject_identifier, report_datetime=onschedule_datetime):

        put_on_schedule(onschedule_model=onschedule_model,
                        schedule_name=schedule_name,
                        onschedule_datetime=onschedule_datetime,
                        base_appt_datetime=onschedule_datetime,
                        subject_identifier=subject_identifier)

        delete_completed_appointments(
                appointment_model_cls=appointment_cls,
                subject_identifier=subject_identifier,
                schedule_name=schedule_name) 
            
    return onschedule_model, schedule_name

In [57]:
def get_onschedule_obj(subject_identifier, cohort):
    cohort = f'{cohort}_' if 'sec' not in cohort else cohort
    try:
        schedule_obj = SubjectScheduleHistory.objects.filter(
            subject_identifier=subject_identifier,
            schedule_name__icontains=cohort.replace('cohort_', '')).filter(
            Q(schedule_name__icontains='qt') | Q(schedule_name__icontains='quart')).latest('onschedule_datetime')
    except SubjectScheduleHistory.DoesNotExist:
        raise
    else:
        return schedule_obj

In [58]:
sidx = 'B142-040990096-5'
child_sidxs = ['B142-040990096-5-25', 'B142-040990096-5-35']

In [ ]:
# consent = SubjectConsent.objects.filter(subject_identifier=sidx).latest('consent_datetime')
# onschedule_datetime = consent.consent_datetime

for idx in child_sidxs:
    try:
        details = CohortSchedules.objects.get(
            schedule_type='followup',
            onschedule_model__startswith='flourish_child',
            cohort_name='cohort_c')
    except CohortSchedules.DoesNotExist:
        print('welleh')
        break
    else:
        onschedule_datetime = OnScheduleChildCohortBFU.objects.get(subject_identifier=idx).onschedule_datetime
        put_onschedule(
            onschedule_datetime=onschedule_datetime,
            subject_identifier=idx,
            onschedule_model=details.onschedule_model,
            schedule_name=details.schedule_name,
            appointment_cls=ChildAppointment)

In [59]:
def update_forms_visit(forms, visit=None, new_visit=None, form_type='crf'):
    for form in forms:
        kwargs = {}
        model_cls = django_apps.get_model(form.model)
        if form_type == 'requisition':
            kwargs = {'panel__name': form.panel_name}
            post_save.disconnect(dispatch_uid='senaite_sample_create_on_post_save')
        try:
            model_obj = model_cls.objects.get(child_visit=visit, **kwargs)
        except model_cls.DoesNotExist:
            continue
        else:
            print(model_obj)
            model_obj.child_visit = new_visit
            model_obj.save()

In [60]:
fu_schedules = [(OnScheduleChildCohortBFU, OnScheduleChildCohortCFU),
                (OnScheduleChildCohortBFUQuart, OnScheduleChildCohortCFUQuart)]
appt_list = ['appt_type', 'appt_status', 'appt_reason', 'comment', 'appt_datetime']
for idx in child_sidxs:
    for schedule_cls in fu_schedules:
        b_schedule_cls, c_schedule_cls = schedule_cls
        try:
            onsch = b_schedule_cls.objects.get(subject_identifier=idx)
        except b_schedule_cls.DoesNotExist:
            print('wheet', b_schedule_cls, idx)
            continue
        else:
            try:
                onsch_c = c_schedule_cls.objects.get(subject_identifier=idx)
            except c_schedule_cls.DoesNotExist:
                print('wheet', c_schedule_cls, idx)
                continue
            else:
                complete_appts = ChildAppointment.objects.filter(
                    subject_identifier=idx, schedule_name=onsch.schedule_name, ).exclude(appt_status='new')
                for appt in complete_appts:
                    # Get corresponding appt
                    new_appt = ChildAppointment.objects.filter(subject_identifier=idx,
                                                               visit_code=appt.visit_code,
                                                               visit_code_sequence=appt.visit_code_sequence,
                                                               schedule_name=onsch_c.schedule_name)
                    if new_appt.exists():
                        appt_dict = {key: getattr(appt, key) for key in appt_list}
                        new_appt.update(**appt_dict)
                    try:
                        visit = ChildVisit.objects.get(appointment=appt)
                    except ChildVisit.DoesNotExist:
                        continue
                    else:
                        new_visit = copy(visit)
                        new_visit.pk = None
                        new_visit.appointment = new_appt[0]
                        new_visit.schedule_name = new_appt[0].schedule_name
                        new_visit.visit_schedule_name = new_appt[0].visit_schedule_name
                        new_visit.save()
                        
                        crfs = CrfMetadata.objects.filter(
                            subject_identifier=idx,
                            visit_code=visit.visit_code,
                            visit_code_sequence=visit.visit_code_sequence,
                            schedule_name=visit.schedule_name,
                            entry_status='KEYED')
                        print(visit, new_visit)
                        update_forms_visit(forms=crfs, visit=visit, new_visit=new_visit)

                        reqs = RequisitionMetadata.objects.filter(
                            subject_identifier=idx,
                            visit_code=visit.visit_code,
                            visit_code_sequence=visit.visit_code_sequence,
                            schedule_name=visit.schedule_name,
                            entry_status='KEYED')
                        update_forms_visit(forms=reqs, visit=visit, new_visit=new_visit, form_type='requisition')
                        visit.save()

B142-040990096-5-25 3000.0 B142-040990096-5-25 3000.0
997YP32 Lead
https://bhplims-dev.bhp.org.bw/senaite/@@API/senaite/v1/auth
https://bhplims-dev.bhp.org.bw/senaite/@@API/senaite/v1/search?portal_type=Client&Title=Flourish
https://bhplims-dev.bhp.org.bw/senaite/@@API/senaite/v1/search?portal_type=Contact&getParentUID=b44782f50fec4a7fbc19f654025d0aba&getFullname=Sam+Kgole
https://bhplims-dev.bhp.org.bw/senaite/@@API/senaite/v1/search?portal_type=SampleType&catalog=bika_setup_catalog&title=Whole+Blood+EDTA
https://bhplims-dev.bhp.org.bw/senaite/@@API/senaite/v1/search?portal_type=ContainerType&catalog=bika_setup_catalog&Title=EDTA+tube
https://bhplims-dev.bhp.org.bw/senaite/@@API/senaite/v1/search?portal_type=ARTemplate&catalog=bika_setup_catalog&getClientUID=b44782f50fec4a7fbc19f654025d0aba&title=Lead
https://bhplims-dev.bhp.org.bw/senaite/@@API/senaite/v1/search?portal_type=Courier&portal_type=ClientCourier&getParentUID=b44782f50fec4a7fbc19f654025d0aba&Title=
More than one object fou

In [ ]:
RequisitionMetadata.objects.filter(
    subject_identifier='B142-040990096-5-25', visit_code='3000', schedule_name='child_b_fu_schedule1').delete()
CrfMetadata.objects.filter(
    subject_identifier='B142-040990096-5-25', visit_code='3001', schedule_name='child_b_fu_qt_schedule1').delete()

In [66]:
for idx in child_sidxs:
    for schedule_cls in fu_schedules:
        b_schedule_cls, c_schedule_cls = schedule_cls
        try:
            onsch = b_schedule_cls.objects.get(subject_identifier=idx)
        except b_schedule_cls.DoesNotExist:
            continue
        else:
            ssh = SubjectScheduleHistory.objects.filter(
                subject_identifier=idx, schedule_name=onsch.schedule_name)
            appts = ChildAppointment.objects.filter(subject_identifier=idx, schedule_name=onsch.schedule_name)
            visits = ChildVisit.objects.filter(appointment__in=appts)
            if ssh.exists():
                ssh.delete()
            for visit in visits:
                visit.delete()
            appts.delete()
            onsch.delete()

In [71]:
onsch = OnScheduleCohortBQuarterly.objects.get(
    subject_identifier=sidx, child_subject_identifier=child_sidxs[0])
ssh = SubjectScheduleHistory.objects.get(subject_identifier=sidx, schedule_name=onsch.schedule_name)

try:
    details = CohortSchedules.objects.get(
        schedule_type='followup',
        onschedule_model__startswith='flourish_caregiver',
        cohort_name='cohort_c',
        child_count=1)
except CohortSchedules.DoesNotExist:
    print('welleh')
else:
    onschedule_datetime = OnScheduleChildCohortCFU.objects.get(
        subject_identifier=child_sidxs[0]).onschedule_datetime
    take_off_schedule(ssh, offschedule_dt=onschedule_datetime)
    put_onschedule(
        onschedule_datetime=onschedule_datetime,
        subject_identifier=sidx,
        onschedule_model=details.onschedule_model,
        schedule_name=details.schedule_name,
        appointment_cls=Appointment)

    update_onschedule_model(
        subject_identifier=sidx, child_subject_identifier=child_sidxs[0],
        onschedule_model=details.onschedule_model, schedule_name=details.schedule_name,)